将数据库中的单人单日资金流动按账户合并：<br/>
* whole 所有投资者
* institution 所有机构投资者
* individual 所有个人投资者
* momentum 所有动量型
* reverse 所有逆向型

In [1]:
import json
import pickle
import pymysql

In [2]:
def merge(cfs):
    d = {}
    for cf in cfs:
        for v1,v2,w in cf:
            if (v1,v2) in d:
                d[(v1,v2)] += w
            else:
                d[(v1,v2)] = w
    result = []
    for v1,v2 in d.keys():
        result.append([v1,v2,d[(v1,v2)]])
    return result

In [3]:
conn = pymysql.connect(
    host = '219.224.169.45',
    user = 'lizimeng',
    password = 'codegeass',
    db = 'capital_flow',
    charset = 'utf8'
)

with conn.cursor() as cursor:
    whole = []
    institution = []
    individual = []
    momentum = []
    reverse = []
    
    cursor.execute("select day from market.trade_day where day between '20140101' and '20161231' \
    order by day asc")
    days = [x for x, in cursor.fetchall()]
    
    for day in days[1:]:
        # whole
        cursor.execute("select cf from basic where date=%s", day)
        whole.append(merge([json.loads(cf) for cf, in cursor.fetchall()]))
        
        # institution
        cursor.execute("select cf from basic where date=%s and account in (select account from investor.zhanghu where account_head_type='3')", day)
        institution.append(merge([json.loads(cf) for cf, in cursor.fetchall()]))
        
        # individual
        cursor.execute("select cf from basic where date=%s and account in (select account from investor.zhanghu where account_head_type='0')", day)
        individual.append(merge([json.loads(cf) for cf, in cursor.fetchall()]))
        
        # momentum
        cursor.execute("select cf from basic where date=%s and account in (select account from traderType.exp1 where coef>0 and p<=0.1)", day)
        momentum.append(merge([json.loads(cf) for cf, in cursor.fetchall()]))
        
        # reverse
        cursor.execute("select cf from basic where date=%s and account in (select account from traderType.exp1 where coef<0 and p<=0.1)", day)
        reverse.append(merge([json.loads(cf) for cf, in cursor.fetchall()]))
    
    for fname,data in (('whole',whole), ('institution',institution), ('individual',individual), ('momentum',momentum), ('reverse',reverse)):
        with open(fname,'wb') as f:
            pickle.dump(data, f)
    
conn.close()
